In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

import pyproj
from pyproj import Geod
from pyproj import Proj, transform
from fiona.crs import from_epsg
from osgeo import osr
from scipy import interpolate

# First create a dataset of M2 smb climate

In [2]:
# ds = xr.open_dataset("/scratch/summit/erke2265/surface-height-change/data/M2_1980-2020_daily_SMB.nc")
# M2_smb_climate = ds["__xarray_dataarray_variable__"].sum(dim='time') / (2020 - 1980 + 1)
# M2_smb_climate.rename(new_name_or_name_dict="Annual-mean-SMB")
# M2_smb_climate.to_netcdf("M2_annual-mean-SMB.nc")

# Copy Brooke RF from petalib to scratch

In [3]:
# !cp /pl/active/nasa_smb/Data/RF_Snow_Red_prelim.nc RF_Snow_Red_prelim.nc

# Function to Random forest SMB ratio

In [4]:
# Function to convert lat/lon to epsg3031
def lat_lon_to_epsg3031(tgt_lat, tgt_lon):
    
    # Source and target EPSG
    src = osr.SpatialReference()
    tgt = osr.SpatialReference()
    src.ImportFromEPSG(4326) # WGS-84
    tgt.ImportFromEPSG(3031) # South Polar Stereo
    
    # Define transformation
    transform = osr.CoordinateTransformation(src, tgt)
    
    # Perform transformation
    coords = transform.TransformPoint(tgt_lat, tgt_lon)
    tgt_x, tgt_y = coords[0:2]
    
    return tgt_x, tgt_y

# Function to get ratio at lat/lon
def get_RF_M2_ratio(tgt_lat, tgt_lon):
    
    # Get M2 SMB at lat/lon
    M2_smb = xr.open_dataset("M2_annual-mean-SMB.nc")
    M2_smb = M2_smb['__xarray_dataarray_variable__']
    M2_smb = M2_smb.sel(lat=tgt_lat, lon=tgt_lon, method='nearest').values
    
    # Get RF at lat/lon
    tgt_x, tgt_y = lat_lon_to_epsg3031(tgt_lat, tgt_lon)
    M2_RF = xr.open_dataset("RF_Snow_Red_prelim.nc")
    M2_RF = M2_RF['IS2_opt'].sel(x=tgt_x, y=tgt_y, method='nearest').values
    
    # Get ratio
    RF_ratio = (M2_RF + M2_smb) / M2_smb
    
    # Print info
    print("Random Forest Ratio: " + str(RF_ratio))
    print("MERRA-2 SMB [mm/yr]: " + str(M2_smb))
    print("Random Forest Perturbation [mm/yr]: " + str(M2_RF))
    
    return RF_ratio, M2_smb, M2_RF


# Test

In [5]:
%%time
tgt_lat = -75
tgt_lon = 90
RF_ratio, M2_smb, M2_RF = get_RF_M2_ratio(tgt_lat, tgt_lon)

Random Forest Ratio: 1.3134992510986843
MERRA-2 SMB [mm/yr]: 55.05298
Random Forest Perturbation [mm/yr]: 17.25906753540039
CPU times: user 25.2 ms, sys: 5.6 ms, total: 30.8 ms
Wall time: 92.2 ms
